In [87]:
using LinearAlgebra
using SparseArrays
include("bArnoldi.jl")
include("arnoldi.jl")

arnoldi (generic function with 1 method)

In [88]:
n = 100
M = 30
L = 3
# We need this condition for now
M * L <= n

true

In [89]:
#^^^^^^^^^^^^^^^^^^^^^^^#
#  A*X_σ - X_σ * D = B  #
#_______________________#
# LHS
#   A
λ = [i+10 for i in 1:n]
a = randn(n, n)
A = UpperTriangular(a) - diagm(diag(a)) + diagm(λ)
#   Shift
σ = rand(1:9, L)
D = diagm(σ) 
Γ = .7
# RHS
B = randn(n, L);

In [90]:
X_exact = zeros(n, L)
for l=1:L
    lhs = A # - D[l, l]*I
    b = B[:, l]
    X_exact[:, l] = lhs \ b
end

In [91]:
# Naive: Standard Arnoldi in a for-loop
X_σ_naive = zeros(n, L) 
@time begin
    for l=1:L
        #print("\nsystem $l \n")
        lhs = A # - D[l, l]*I
        b = B[:, l]
        Q, H = arnoldi(lhs, b, M)
        for m in 1:M # Changed the following to be consistent with bArnoldi and gives smaller norms
            #print("m$m\t")
            s = vcat(norm(b), zeros(m-1, 1)) #   size m+1 -> m
            z = H[1:m, 1:m] \ s              #   H[1:m+1, 1:m] -> H[1:m, 1:m]
            x = Q[:, 1:m] * z
            X_σ_naive[:, l] = x
        end
        #print("\n\n")
    end
end # time

  0.049560 seconds (56.07 k allocations: 9.147 MiB, 94.22% compilation time)


In [92]:
# Block Arnoldi
X_σ = zeros(n, L) 
Res = zeros(n, L)
@time begin
    W, H = bArnoldi(A, B, M)
    f = qr(B)
    for m in 1:M
        #print("m$m\t")
        s = f.Q[:, 1:m*L]' * B   # s = Q' B
        z = H[1:L*m, 1:L*m] \ s  # z = H \ s
        x = W[:, 1:L*m] * z      # x = W_m z    
        X_σ = x
    end
    #print("\n")
end # time

  0.056011 seconds (99.53 k allocations: 17.816 MiB, 91.07% compilation time)


In [93]:
norm(X_σ - X_σ_naive), norm(X_exact - X_σ), norm(X_exact - X_σ_naive)

(1.8582760814184719e-9, 3.5752763904468557e-13, 1.8583020538270672e-9)